In [ ]:
using GeoDataFrames
using DataFrames
using Plots
pyplot()
using Shapefile
using Random
using ArchGDAL
using GeoInterface

In [ ]:
###Shapefile municipalities

shapefile_path = "C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Análises\\MT_Municipios_2022\\MT_Municipios_2022.shp"
municipalities = Shapefile.Table(shapefile_path) |> DataFrame

In [ ]:
# Lista de municípios para filtrar
municipios_filtrados = [
    "Cuiabá", "Várzea Grande"]

# Filtrando os dados
filtered_municipalities = filter(row -> row.NM_MUN in municipios_filtrados, municipalities)

# Exibindo os dados filtrados
println(filtered_municipalities)

In [ ]:
# Highway shapefile
shapefile_path_highways = "C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Análises\\Rodovias\\SNV_202410A.shp"
highways = Shapefile.Table(shapefile_path_highways) |> DataFrame

In [ ]:
# Filtering sg_uf equals to MT
filtered_highways_MT = filter(row -> row.sg_uf == "MT", highways)

# Grouping by vl_br row
grouped_highways = groupby(filtered_highways_MT, :vl_br)

In [ ]:
#Combine geometries
function combine_geometries(geometries)

    return mapfoldl(
        x -> GeoInterface.convert(ArchGDAL, x), # map - apply to each element
        ArchGDAL.union, # reduce 2 elements to 1 element
        geometries # the vector to perform the operation on
    )
end

In [ ]:
# Applying function
combined_geometries = combine(grouped_highways, :geometry => combine_geometries => :geometry)

#Showing results
println(combined_geometries)

In [ ]:
# Gerar cores aleatórias para cada geometria
colors = [RGB(rand(), rand(), rand()) for _ in 1:length(combined_geometries.geometry)]

In [ ]:
# Definir cores para municipios_filtrados
colors2 = [:black, :gray]

In [ ]:
# Plotar todos os municípios e as rodovias filtradas
plot(size=(1000, 800))  # Definir tamanho do gráfico
plot!(legend=:outerright)
plot!(municipalities.geometry, color=:white, linecolor=:black, label=false)

In [ ]:
# Plotar os municípios filtrados com cores específicas
for (i, NM_MUN) in enumerate(filtered_municipalities.geometry)
    plot!(NM_MUN, color=colors2[i], label=municipios_filtrados[i])
end

In [ ]:
# Plotar as rodovias filtradas
for (i, geometry) in enumerate(combined_geometries.geometry)
    plot!(geometry, color=colors[i], linewidth=3, label="BR-" * string(combined_geometries.vl_br[i]))
end

In [ ]:
xlabel!("")
ylabel!("")
title!("")
plot!(legend=:outerright, legendfontsize=8)  # Colocar a legenda ao lado e diminuir o tamanho da legenda
plot!(framestyle=:none, grid=false)  # Excluir eixos e marcas de quadrado

In [ ]:
# Salvar o gráfico
savefig("C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Dissertação\\Quadros e gráficos\\Graph- Hub Cuiabá e Várzea Grande.png")